<a href="https://colab.research.google.com/github/LuisFernandoPareja/Portfolio/blob/main/FIFA_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FIFA 21 messy, raw dataset for cleaning/ exploring
The purpose of this project is to clean and transform data from a raw dataset of EA Sports' latest installment of their hit FIFA series - FIFA21, which was scraped from sofifa.com by Rachit Toshniwal (Dataset Available on Kaggle [here](https://www.kaggle.com/datasets/yagunnersya/fifa-21-messy-raw-dataset-for-cleaning-exploring)).

This dataset simulates real-world situations when scraping data from the web. You will encounter almost surely unclean data with wrong data types and this project aims to demonstrate programming techniques to correct this.

We will start by obtaining the dataset from Kaggle through the API (The following steps are to do so in Google Colab which is the enviroment I used)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Upload Files
from google.colab import files

files.upload() #this will prompt you to upload the kaggle.json

!ls -lha kaggle.json

# Install kaggle API client
!pip install -q kaggle

# kaggle API client expects the file to be in ~/.kaggle
# so move it there
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Now all you need is to set permissions to the json file
!chmod 600 /root/.kaggle/kaggle.json

# check your directory before downloading the datasets
!pwd #Print Working Directory

# Now everything is ready, you can list all available datasets
#!kaggle datasets list

#!/bin/bash
!kaggle datasets download yagunnersya/fifa-21-messy-raw-dataset-for-cleaning-exploring

#Now all you have to do is extract the contents of the zip file
!unzip fifa-21-messy-raw-dataset-for-cleaning-exploring.zip

In [ ]:
#Move the dataset to the drive so you wont have to download this every time
!mv "/content/fifa21_raw_data.csv" '/content/gdrive/MyDrive/Datasets'

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Imports
Since we are only going to clean and transform data, we'll only use pandas



In [4]:
import pandas as pd

Now we're going to take a look at the dataset and see if everything is in order.
The FIFA 21 dataset contains information about players, their clubs, salaries, individual stats, etc.

In [5]:
fifa_dataset = pd.read_csv("/content/gdrive/MyDrive/Datasets/fifa21_raw_data.csv")
fifa_dataset.head()

<ipython-input-5-87930fc9631a>:1: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa_dataset = pd.read_csv("/content/gdrive/MyDrive/Datasets/fifa21_raw_data.csv")


,photoUrl,LongName,playerUrl,Nationality,Positions,Name,Age,↓OVA,POT,Team & Contract,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,https://cdn.sofifa.com/players/158/023/21_60.png,Lionel Messi,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,RW ST CF,L. Messi,33,93,93,\n\n\n\nFC Barcelona\n2004 ~ 2021\n\n,...,Medium,Low,5 ★,85,92,91,95,38,65,\n372
1,https://cdn.sofifa.com/players/020/801/21_60.png,C. Ronaldo dos Santos Aveiro,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,ST LW,Cristiano Ronaldo,35,92,92,\n\n\n\nJuventus\n2018 ~ 2022\n\n,...,High,Low,5 ★,89,93,81,89,35,77,\n344
2,https://cdn.sofifa.com/players/200/389/21_60.png,Jan Oblak,http://sofifa.com/player/200389/jan-oblak/210005/,Slovenia,GK,J. Oblak,27,91,93,\n\n\n\nAtlético Madrid\n2014 ~ 2023\n\n,...,Medium,Medium,3 ★,87,92,78,90,52,90,\n86
3,https://cdn.sofifa.com/players/192/985/21_60.png,Kevin De Bruyne,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,CAM CM,K. De Bruyne,29,91,91,\n\n\n\nManchester City\n2015 ~ 2023\n\n,...,High,High,4 ★,76,86,93,88,64,78,\n163
4,https://cdn.sofifa.com/players/190/871/21_60.png,Neymar da Silva Santos Jr.,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,LW CAM,Neymar Jr,28,91,91,\n\n\n\nParis Saint-Germain\n2017 ~ 2022\n\n,...,High,Medium,5 ★,91,85,86,94,36,59,\n273


We will tackle this dataset problem with a step by step approach with questions.
We can see that the data is messy. We're going to do some extra verifications to check that everything is in order like data types and what not.


## 1. We are going to start by checking the height and weight columns, do these columns have the appropiate data types?

In [6]:
fifa_dataset['Height'].dtypes

dtype('O')

In [7]:
fifa_dataset['Weight'].dtypes

dtype('O')

We would like to have the height and weight columns in a numerical data type but they are string (Additionally we would like to also have them in kg and cm).

In [8]:
def height_to_cm(height):
  # Remove the ' and " symbols
  height = height.replace("'", "").replace('"', "")
  # Extract feet and inches
  feet = int(height[0])
  inches = int(height[1:])

  # Convert feet and inches to centimeters
  return feet * 30.48 + inches * 2.54

def weight_to_kg(weight):
  # Remove the ' and " symbols
  weight = weight.replace("lbs", "")
  # Convert weight to kilograms
  return float(weight) * 0.453592

In [9]:
# Apply the previously created functions to the Height_cm and Weight_kg columns.
fifa_dataset['Height_cm'] = fifa_dataset['Height'].apply(height_to_cm)
fifa_dataset['Weight_kg'] = fifa_dataset['Weight'].apply(weight_to_kg)

In [10]:
# Visualize the new columns
fifa_dataset[['Height', 'Height_cm', 'Weight', 'Weight_kg']]

,Height,Height_cm,Weight,Weight_kg
0,"5'7""",170.18,159lbs,72.121128
1,"6'2""",187.96,183lbs,83.007336
2,"6'2""",187.96,192lbs,87.089664
3,"5'11""",180.34,154lbs,69.853168
4,"5'9""",175.26,150lbs,68.038800
...,...,...,...,...
18974,"5'10""",177.80,154lbs,69.853168
18975,"5'7""",170.18,128lbs,58.059776
18976,"5'10""",177.80,146lbs,66.224432
18977,"5'9""",175.26,143lbs,64.863656


In [11]:
# Drop the old columns since the ones we just created are more useful
fifa_dataset = fifa_dataset.drop(['Height', 'Weight'], axis=1)

In [12]:
fifa_dataset[['Height_cm', 'Weight_kg']]

,Height_cm,Weight_kg
0,170.18,72.121128
1,187.96,83.007336
2,187.96,87.089664
3,180.34,69.853168
4,175.26,68.038800
...,...,...
18974,177.80,69.853168
18975,170.18,58.059776
18976,177.80,66.224432
18977,175.26,64.863656


## 2. Can we separate the 'Joined' column into 'Year', 'Month' and 'Day' columns?

In [13]:
fifa_dataset['Joined'].head()

,Joined
0,"Jul 1, 2004"
1,"Jul 10, 2018"
2,"Jul 16, 2014"
3,"Aug 30, 2015"
4,"Aug 3, 2017"


In [14]:
# Convert the date column to datetime format
fifa_dataset['Joined'] = pd.to_datetime(fifa_dataset['Joined'])
fifa_dataset['Joined'].head()

,Joined
0,2004-07-01
1,2018-07-10
2,2014-07-16
3,2015-08-30
4,2017-08-03


In [15]:
# Here we split the date into year, month and day. Pandas helps us with that
fifa_dataset['Year_Joined'] = fifa_dataset['Joined'].dt.year
fifa_dataset['Month_Joined'] = fifa_dataset['Joined'].dt.month
fifa_dataset['Day_Joined'] = fifa_dataset['Joined'].dt.day

In [16]:
fifa_dataset[['Year_Joined', 'Month_Joined', 'Day_Joined']].head()

,Year_Joined,Month_Joined,Day_Joined
0,2004,7,1
1,2018,7,10
2,2014,7,16
3,2015,8,30
4,2017,8,3


## 3. Now, Clean and Transform the 'Value', 'Wage' and 'Release Clause' columns into columns of integers

In [17]:
fifa_dataset[['Value', 'Wage', 'Release Clause']].head()

,Value,Wage,Release Clause
0,€67.5M,€560K,€138.4M
1,€46M,€220K,€75.9M
2,€75M,€125K,€159.4M
3,€87M,€370K,€161M
4,€90M,€270K,€166.5M


We can see that these values contain symbols and letters at the beginning and at the end, first we get rid of these and then we transform to integer.

In [18]:
def remove_symbols(value):
  if 'M' in value:
    # Get rid of the undesired symbols using the strip() function
    value = value.strip('€M')
    # Since the value contains the letter M it means that it has to be multiplied by one million
    value = float(value) * 1_000_000
  elif 'K' in value:
    # Get rid of the undesired symbols using the strip() function
    value = value.strip('€K')
    # Since the value contains the letter K it means that it has to be multiplied by a thousand
    value = float(value) * 1000
  else:
    value = value.strip('€')
  return int(value)

In [19]:
# Apply the previously created functions to the data and replace.
fifa_dataset['Value'] = fifa_dataset['Value'].apply(remove_symbols)
fifa_dataset['Wage'] = fifa_dataset['Wage'].apply(remove_symbols)
fifa_dataset['Release Clause'] = fifa_dataset['Release Clause'].apply(remove_symbols)

In [20]:
fifa_dataset[['Value', 'Wage', 'Release Clause']]

,Value,Wage,Release Clause
0,67500000,560000,138400000
1,46000000,220000,75900000
2,75000000,125000,159400000
3,87000000,370000,161000000
4,90000000,270000,166500000
...,...,...,...
18974,35000,1000,57000
18975,60000,500,165000
18976,40000,1000,70000
18977,60000,500,165000


## Remove the newline characters from the 'Hits' column

In [21]:
fifa_dataset['Hits'].head()

,Hits
0,\n372
1,\n344
2,\n86
3,\n163
4,\n273


In [22]:
# Since the newline characters are at the beginning we use the strip() function
fifa_dataset['Hits'] = fifa_dataset['Hits'].str.strip('\n')

In [23]:
fifa_dataset['Hits'].head()

,Hits
0,372
1,344
2,86
3,163
4,273


## 4. Should you separate the 'Team & Contract' column into separate 'Team' and 'Contract' columns?

In my opinion I say **yes** for different reasons:

**Data Organization:**

Having separate columns for "Team" and "Contract" improves the structure of the dataset, making it easier to query, filter, and analyze.

**Readability:**

It is easier to read and interpret the data when the team names and contract durations are in their own columns.

**Data Analysis:**

Analyze trends based on the team or contract durations independently. For example:
Filtering players by their current team.
Calculating average contract lengths by team.
With one combined column, these type of analyses would require additional processing.

**Normalization:**

Separating "Team" and "Contract" into distinct columns aligns with the principles of database normalization..




In [24]:
fifa_dataset['Team & Contract'].head()

,Team & Contract
0,\n\n\n\nFC Barcelona\n2004 ~ 2021\n\n
1,\n\n\n\nJuventus\n2018 ~ 2022\n\n
2,\n\n\n\nAtlético Madrid\n2014 ~ 2023\n\n
3,\n\n\n\nManchester City\n2015 ~ 2023\n\n
4,\n\n\n\nParis Saint-Germain\n2017 ~ 2022\n\n


In [25]:
# Split into 'Team' and 'Contract Period'
fifa_dataset['Team & Contract'] = fifa_dataset['Team & Contract'].str.strip('\n')
fifa_dataset[['Team', 'Contract Period']] = fifa_dataset['Team & Contract'].str.split('\n', expand=True)

# Drop the original column
fifa_dataset = fifa_dataset.drop(columns=['Team & Contract'])

In [26]:
fifa_dataset[['Team', 'Contract Period']]

,Team,Contract Period
0,FC Barcelona,2004 ~ 2021
1,Juventus,2018 ~ 2022
2,Atlético Madrid,2014 ~ 2023
3,Manchester City,2015 ~ 2023
4,Paris Saint-Germain,2017 ~ 2022
...,...,...
18974,Chongqing Dangdai Lifan FC SWM Team,2020 ~ 2020
18975,Oldham Athletic,2020 ~ 2021
18976,Wuhan Zall,2018 ~ 2022
18977,Oldham Athletic,2020 ~ 2021


Now the Contract Period column seems a little bit weird to have it like that, so what we're going to do is to separate this column into 'Start Year' and 'End Year'.

In [27]:
fifa_dataset[['Start Year', 'End Year']] = fifa_dataset['Contract Period'].str.split('~', expand=True)

In [28]:
fifa_dataset = fifa_dataset.drop(columns=['Contract Period'])

In [29]:
fifa_dataset[['Start Year', 'End Year']]

,Start Year,End Year
0,2004,2021
1,2018,2022
2,2014,2023
3,2015,2023
4,2017,2022
...,...,...
18974,2020,2020
18975,2020,2021
18976,2018,2022
18977,2020,2021


## 5. There's star ★ symbols in the columns 'W/F','SM' & 'IR'.
This is not very useful so we're going to remove them and transform these columns data type from string to int

In [30]:
# Strip the '★' symbol
fifa_dataset['W/F'] = fifa_dataset['W/F'].str.strip('★')
fifa_dataset['SM'] = fifa_dataset['SM'].str.strip('★')
fifa_dataset['IR'] = fifa_dataset['IR'].str.strip('★')

In [31]:
fifa_dataset[['W/F', 'SM', 'IR']]

,W/F,SM,IR
0,4,4,5
1,4,5,5
2,3,1,3
3,5,4,4
4,5,5,5
...,...,...,...
18974,2,2,1
18975,2,2,1
18976,2,2,1
18977,2,2,1


And now that we finished our data cleaning and transformation of this raw data, we can now create a new csv file with all these changes.

In [32]:
fifa_dataset.to_csv('fifa_out.csv', index=False)